### TRATAMENTO 1

Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\LiquidadosBaixados

Arquivo:
- 20250218_172853_TituloLiquidadosBaixados_BRISTOLFIDCMULTISSETORIAL.csv

Tratamentos:
1. Converter notação ciêntifica para número coluna (NUMERO CORRESPONDENTE)
2. Alterar para data abreviada coluna (DATA MOVIMENTO)

### TRATAMENTO 2

Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\LiquidadosBaixados

Arquivo:
- 20250218_181859_LiquidadosEBaixados_ÁGISFIDC_001.csv

Tratamento
1. Excluir linhas a direita


In [2]:
import os
import time
import pandas as pd

from datetime import datetime, timedelta
datetime.today().strftime('Data: %Y-%m-%d Hora: %H:%m')

'Data: 2025-03-24 Hora: 09:03'

# DAYCOVAL

In [3]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_agis = list(filter(lambda x: any(item in x for item in ['ÁGISFIDC','CREDIAL','AGISFIDCNAOPADRONIZADO-TRANSFERIDO','AGISFIDCNAOPADRONIZADO']), arquivos)) # 

stop = 0
frames = []

while stop < len(csv_agis):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_agis[stop]}'
    df_csv_agis = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_agis)
    stop += 1

df_combined_agis = pd.concat(frames, ignore_index=True)

In [4]:
df_combined_agis.FUNDO.drop_duplicates()

0    ÁGIS FIDC
Name: FUNDO, dtype: object

In [5]:
# df_csv_agis = df_combined_agis[['FUNDO','CEDENTE','CPF/CNPJ','SACADO','CPF/CNPJ.1','SEU NUMERO','DOCUMENTO','TIPO RECEBIVEL',\
#                   'VALOR DE AQUISICÃO','VALOR DE PAGO','VALOR DE VENCIMENTO','AJUSTE','SITUACÃO DO RECEBÃVEL','DATA MOVIMENTO','DATA DA AQUISIÃÃO','DATA DE VENCIMENTO',\
#                   'TAXA','OCORRÃNCIA']] ## quando for historico

df_csv_agis = df_combined_agis[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO','SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',\
                  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE','ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',\
                  'TX_AQUISICAO','TIPO_MOVIMENTO']]

## renomeia colunas
df_csv_agis.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_agis[date_columns] = df_csv_agis[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_agis[column] = df_csv_agis[column].apply(transformar_valor)
    
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'ÁGIS FIDC', 'NmFundo'] = 'FIDC AGIS'
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'
df_csv_agis.loc[df_csv_agis['NmFundo'] == 'CREDIAL BANK PAN FIDC', 'NmFundo'] = 'FIDC CREDIAL'


# df_csv_agis[['NmFundo','VlrAquisicao', 'VlrPago','VlrVencimento', 'TaxaAquisicao']].head()
df_csv_agis[['NmFundo','VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']].head()


C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\2380470613.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_agis[date_columns] = df_csv_agis[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\2380470613.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_agis[column] = df_csv_agis[column].apply(transformar_valor)


,NmFundo,VlrAquisicao,VlrVencimento,VlrPago,VlrAjuste,TaxaAquisicao
0,FIDC AGIS,7132.45,7272.80,7364.92,-92.12,0.278300
1,FIDC AGIS,6783.98,6909.10,7005.85,-96.75,0.258940
2,FIDC AGIS,81055.63,86622.00,86622.00,0.00,0.334525
3,FIDC AGIS,127423.95,136925.43,136925.43,0.00,0.339514
4,FIDC AGIS,20201.63,21100.00,21100.00,0.00,1.324302


In [6]:
df_csv_agis[['NmFundo','DtMovimento']].drop_duplicates()

,NmFundo,DtMovimento
0,FIDC AGIS,2025-03-20


In [7]:
df_csv_agis['NmFundo'].count()

228

___________________

# AGROFORTE - APG

In [8]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_outros2 = list(filter(lambda x: any(item in x for item in ['AGROFORTEFIDC','APGFIDC', 'BRISTOLFIDCMULTISSETORIAL']), arquivos)) # 

stop = 0
frames = []

while stop < len(csv_outros2):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_outros2[stop]}'
    df_csv_outros2 = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_outros2)
    stop += 1

df_combined_outros2 = pd.concat(frames, ignore_index=True)
df_combined_outros2

,FUNDO,DATA MOVIMENTO,CEDENTE,CPF/CNPJ,OCORRENCIA,SITUACAO DO RECEBIVEL,DOCUMENTO,SACADO,CPF/CNPJ.1,TAXA,ID_RECEBIVEL,VALOR DE AQUISICAO,VALOR DE VENCIMENTO,DATA DA AQUISICAO,DATA DE VENCIMENTO,VALOR DE PAGO,AJUSTE,SEU NUMERO,NUMERO CORRESPONDENTE,TIPO RECEBIVEL
0,APG FIDC,20/03/2025,COMFRIO SOLUCOES LOGISTICAS SA,01.413.969/0001-57,BAIXA POR DEPOSITO SACADO,VENCIDOS,1164,BIMBO DO BRASIL LTDA,35.402.759/0111-10,"0,4307849859",334133141,"103435,93","107330,40",07/02/2025,19/03/2025,107330.40,"0,00",0000000449650000000012413,NaN,Duplicata de Servico Fisico
1,APG FIDC,20/03/2025,COMFRIO SOLUCOES LOGISTICAS SA,01.413.969/0001-57,BAIXA POR DEPOSITO SACADO,VENCIDOS,1161,CITROSUCO S/A AGROINDUSTRIA,33.010.786/0001-87,"0,4307858808",334133140,"137027,59","142186,83",07/02/2025,19/03/2025,142186.83,"0,00",0000000449650000000012416,NaN,Duplicata de Servico Fisico
2,APG FIDC,20/03/2025,COMFRIO SOLUCOES LOGISTICAS SA,01.413.969/0001-57,BAIXA POR DEPOSITO SACADO,VENCIDOS,1159,VIGOR ALIMENTOS S.A,13.324.184/0030-21,"0,4307843842",334133139,"54777,84","56840,28",07/02/2025,19/03/2025,56840.28,"0,00",0000000449650000000012414,NaN,Duplicata de Servico Fisico
3,APG FIDC,20/03/2025,GOOD SERV DE CLIMATIZACAO LTDA,04.437.658/0001-16,BAIXA POR DEPOSITO SACADO,VENCIDOS,10064-5,CWB SMART ASSIST TEC LTDA.,44.988.920/0001-35,"0,5374081963",327301592,"26444,98","28850,00",02/01/2025,18/03/2025,29457.08,"-607,08",0000000455940000000010759,NaN,Duplicata de Servico Fisico
4,APG FIDC,20/03/2025,MS INCORPORADORA SA,05.289.609/0001-46,LIQUIDAÇÃO NORMAL,VENCIDOS NA DATA,210215,CARLOS ROBERTO DUFFECK MATOS,356.507.709-30,"0,7463027291",337622217,"1228,00","1263,83",27/02/2025,20/03/2025,1263.83,"0,00",0000000451300000000013036,60000004525P,Contrato Fisico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,BRISTOL FIDC MULTISSETORIAL,20/03/2025,METALURGICA SCHWARZ S A,72.242.621/0001-92,BAIXA POR DEPOSITO SACADO,VENCIDOS,95671001,MAHLE COMPRESSORES DO BRASIL LTDA,20.985.558/0001-80,"0,24453204",336379455,"341588,69","345165,66",20/02/2025,12/03/2025,345165.66,0,0000000299730000000200679,NaN,Duplicata
116,BRISTOL FIDC MULTISSETORIAL,20/03/2025,CATIVA TEXTIL INDUSTRIA E COMÉRCIO LTDA,80.959.513/0001-63,LIQUIDAÇÃO NORMAL,À VENCER,1000723001,MALHARIA BAL LTDA,47.207.980/0001-61,"0,372338625",335962434,"5461,33","5621,4",18/02/2025,25/03/2025,5621.40,"-21,14",0000000369200000000200639,504900000244,Duplicata
117,BRISTOL FIDC MULTISSETORIAL,20/03/2025,CATIVA TEXTIL INDUSTRIA E COMÉRCIO LTDA,80.959.513/0001-63,LIQUIDAÇÃO NORMAL,À VENCER,1000724001,MALHARIA BAL LTDA,47.207.980/0001-61,"0,372318257",335962435,"6169,58","6350,4",18/02/2025,25/03/2025,6350.40,"-23,88",0000000369200000000200642,504900000279,Duplicata
118,BRISTOL FIDC MULTISSETORIAL,20/03/2025,CATIVA TEXTIL INDUSTRIA E COMÉRCIO LTDA,80.959.513/0001-63,LIQUIDAÇÃO NORMAL,À VENCER,1003012001,POSTO DA MATA TECIDOS LTDA,00.310.830/0001-15,"0,338278075",338288328,"2110,33","2144,77",05/03/2025,25/03/2025,2144.77,"-7,43",0000000369200000000201939,506400005943,Duplicata


In [9]:
# df_csv_outros2 = df_combined_outros2[['FUNDO','CEDENTE', 'CPF/CNPJ', 'SACADO', 'CPF/CNPJ.1', 'SEU NUMERO', 'DOCUMENTO', 
#                                 'TIPO RECEBIVEL', 'VALOR DE AQUISICAO', 'VALOR DE PAGO', 'VALOR DE VENCIMENTO',  
#                                 'AJUSTE', 'SITUACAO DO RECEBIVEL', 'DATA MOVIMENTO',  'DATA DA AQUISICAO', 
#                                 'DATA DE VENCIMENTO', 'TAXA', 'OCORRENCIA']]
                                


## renomeia colunas
# df_csv_outros2.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
#                        "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
#                        "DtVencimento","TaxaAquisicao","Ocorrencia"]


df_csv_outros2 = df_combined_outros2[['FUNDO','CEDENTE','CPF/CNPJ','SACADO','CPF/CNPJ.1',
                                  'SEU NUMERO','DOCUMENTO','TIPO RECEBIVEL',
                                  'VALOR DE AQUISICAO','VALOR DE PAGO','VALOR DE VENCIMENTO','AJUSTE',
                                  'SITUACAO DO RECEBIVEL','DATA MOVIMENTO','DATA DA AQUISICAO','DATA DE VENCIMENTO',
                                  'TAXA','OCORRENCIA']]
                                

## renomeia colunas
df_csv_outros2.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]


date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_outros2[date_columns] = df_csv_outros2[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_outros2[column] = df_csv_outros2[column].apply(transformar_valor)

df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'AGROFORTE FIDC', 'NmFundo'] = 'FIDC AGROFORTE'
df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'APG FIDC', 'NmFundo'] = 'FIDC APG'
df_csv_outros2.loc[df_csv_outros2['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL', 'NmFundo'] = 'FIDC BRISTOL'
df_csv_outros2['VlrPago'] = df_csv_outros2['VlrPago']/100
df_csv_outros2[['NmFundo','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\4103077885.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_outros2[date_columns] = df_csv_outros2[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\4103077885.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_outros2[column] = df_csv_outros2[column].apply(transformar_valor)
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\4103077885.py:47: SettingWithCopyWarning: 
A 

,NmFundo,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,TaxaAquisicao
0,FIDC APG,103435.93,107330.40,107330.40,0.00,0.430785
1,FIDC APG,137027.59,142186.83,142186.83,0.00,0.430786
2,FIDC APG,54777.84,56840.28,56840.28,0.00,0.430784
3,FIDC APG,26444.98,29457.08,28850.00,-607.08,0.537408
4,FIDC APG,1228.00,1263.83,1263.83,0.00,0.746303


In [10]:
df_csv_outros2.query("NmFundo == 'FIDC AGROFORTE'")[['NmFundo','DtMovimento','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

,NmFundo,DtMovimento,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,TaxaAquisicao


In [11]:
df_csv_outros2.query("NmFundo == 'FIDC BRISTOL'")[['NmFundo','DtMovimento','VlrAquisicao', 'VlrPago','VlrVencimento', 'VlrAjuste','TaxaAquisicao']].head()

,NmFundo,DtMovimento,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,TaxaAquisicao
37,FIDC BRISTOL,2025-03-20,274053.78,292557.74,292557.74,0.0,0.298683
38,FIDC BRISTOL,2025-03-20,265365.45,283282.78,283282.78,0.0,0.298683
39,FIDC BRISTOL,2025-03-20,269251.68,287431.41,287431.41,0.0,0.298683
40,FIDC BRISTOL,2025-03-20,163.26,169.94,169.94,0.0,0.186824
41,FIDC BRISTOL,2025-03-20,1385.35,1442.04,1442.04,0.0,0.186847


In [12]:
df_csv_outros2[['NmFundo','DtMovimento', 'TpRecebivel', 'Ocorrencia']].drop_duplicates().head()

,NmFundo,DtMovimento,TpRecebivel,Ocorrencia
0,FIDC APG,2025-03-20,Duplicata de Servico Fisico,BAIXA POR DEPOSITO SACADO
4,FIDC APG,2025-03-20,Contrato Fisico,LIQUIDAÇÃO NORMAL
7,FIDC APG,2025-03-20,Duplicata,LIQUIDAÇÃO NORMAL
10,FIDC APG,2025-03-20,Duplicata,BAIXA POR RECOMPRA
26,FIDC APG,2025-03-20,Nota Comercial Pre Fixada,BAIXA POR DEPOSITO SACADO


___________________

# Master

In [13]:
df_LiquidadosBaixados = pd.DataFrame()
LiquidadosBaixados = list(filter(lambda x: not any(item in x for item in ['SDL-FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','ÁGIS2FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                                          'ÁGIS', 'AGROFORTEFIDC', 'APGFIDC','K-FINANCEFIDCNP', 'BLUEROCKETFIDC',\
                                                                         'GLOBALFUTURAFIDCRLFIDC','BELL','BRISTOLFIDCMULTISSETORIAL','TORONTOFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','VITTRAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADANIXAUDITORESINDEPENDENTES','3RD-FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','SMTAGROFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','MALBECFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','IPE','BASAFIDCDERESPONSABILIDADELIMITADA',\
                                                                         'ALBAREDOFIDC','VERGINIA','ANVERESFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','USECORPCATÁLISEFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSSEGMENTOCOMERCIALDERESPLIMITADA', 'AF6','CREDIAL','ZINVESTFIDC','VISHNU',\
                                                                         'PrimeAgroFIDCRL','KERDOSFIDC','TERTONFUNDODEINVESTIMENTODIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','NR11FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','RHBCREDFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','SCFIDCRESPONSABILIDADEILIMITADA',\
                                                                         'DBANKFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                                         'VELSO','FUTUROCAPITAL','AGROCETEFIDCNP','PAYCARGOFIDCNP',\
                                                                          'GREENWOODFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSNAOPADRONIZADOS',\
                                                                         'NINE','BONTEMPO','PINPAG','MASTRENN','CREDILOGII-FIDC','F2BANKFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS',\
                                                                         'MFGROUPFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','CAPITALIZAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                                         'UKFFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','BEFICFIDCDERESPONSABILIDADELIMITADA','ANILFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA',\
                                                                         'LIBRACONSIGNADOFIDCDERL', 'LEXCAPITALFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA', 'CONDOBEMFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA']), arquivos))

stop = 0
frames = []

while stop < len(LiquidadosBaixados):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{LiquidadosBaixados[stop]}'
    df_csv_liq = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_liq)
    stop += 1

df_combined_liqu = pd.concat(frames, ignore_index=True)



# df_LiquidadosBaixados = df_combined_liqu[['FUNDO','CEDENTE', 'CPF/CNPJ', 'SACADO', 'CPF/CNPJ.1', 'SEU NUMERO', 'DOCUMENTO', 'TIPO RECEBIVEL', 
# 'VALOR DE AQUISICAO', 'VALOR DE PAGO', 'VALOR DE VENCIMENTO',  'AJUSTE', 'SITUACAO DO RECEBIVEL',
#  'DATA MOVIMENTO',  'DATA DA AQUISICAO', 'DATA DE VENCIMENTO', 'TAXA', 'OCORRENCIA',]]

df_LiquidadosBaixados = df_combined_liqu[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO',
  'SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',
  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE',
  'ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',
  'TX_AQUISICAO','TIPO_MOVIMENTO']]

# renomeia colunas
df_LiquidadosBaixados.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",\
                                 "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao","DtVencimento","TaxaAquisicao","Ocorrencia"]
# Alterando o tipo das colunas de data

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_LiquidadosBaixados[date_columns] = df_LiquidadosBaixados[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao','VlrPago','VlrVencimento','VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_LiquidadosBaixados[column] = df_LiquidadosBaixados[column].apply(transformar_valor)

df_LiquidadosBaixados.loc[df_LiquidadosBaixados['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'


print(df_LiquidadosBaixados[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head().to_string())

        NmFundo  VlrAquisicao  VlrPago  VlrVencimento  VlrAjuste  TaxaAquisicao
0  FIDC VALENTE       8119.71  8287.50        8287.50        0.0       0.278193
1  FIDC VALENTE       3252.98  4060.00        4060.00        0.0       0.293505
2  FIDC VALENTE       3359.57  4060.00        4060.00        0.0       0.269478
3  FIDC VALENTE        547.96   580.34         580.34        0.0       0.277898
4  FIDC VALENTE       1146.80  1170.50        1170.50        0.0       0.278218


C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\345481265.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LiquidadosBaixados[date_columns] = df_LiquidadosBaixados[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\345481265.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LiquidadosBaixados[column] = df_LiquidadosBaixados[column].apply(transformar_valor)


In [14]:
df_LiquidadosBaixados[['NmFundo',"DtMovimento"]].drop_duplicates().sort_values("DtMovimento")

,NmFundo,DtMovimento
0,FIDC VALENTE,2025-03-20
9,FIDC TRACTOR,2025-03-20
29,FIDC TOPCRED,2025-03-20
1755,FIDC PRIMER,2025-03-20
1835,FIDC P2 INVEST,2025-03-20
1850,FIDC FINTRUST,2025-03-20
1873,BR8 FIDC,2025-03-20
1954,FIDC BRAVOS,2025-03-20
2021,FIDC DFC,2025-03-20
2059,FIDC FERPAR,2025-03-20


In [15]:
df_LiquidadosBaixados.loc[
    df_LiquidadosBaixados['NmFundo'].str.contains('YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', case=False, na=False), 
    'NmFundo'
] = 'FIDC YELLOW'

In [16]:
df_LiquidadosBaixados.loc[
    df_LiquidadosBaixados['NmFundo'].str.contains('BR8 FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC BR8'



# Daycoval

In [20]:
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados")

csv_outros = list(filter(lambda x: any(item in x for item in ['SDL-FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','ÁGIS2FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                              'IPE', 'K-FINANCEFIDCNP', 'BLUEROCKETFIDC','GLOBALFUTURAFIDCRLFIDC',\
                                                              'BELLFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','NR11FUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','ANVERESFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','SMTAGROFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','BASAFIDCDERESPONSABILIDADELIMITADA',\
                                                              'ALBAREDOFIDC','VERGINIA','AF6','TORONTOFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA','VITTRAFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADANIXAUDITORESINDEPENDENTES','3RD-FUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','RHBCREDFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','ZINVESTFIDC','VISHNUFIDC',\
                                                              'PrimeAgroFIDCRL','KERDOSFIDC','TERTONFUNDODEINVESTIMENTODIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA','MALBECFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS','SCFIDCRESPONSABILIDADEILIMITADA',\
                                                             'DBANKFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA',\
                                                              'VELSO','FUTUROCAPITAL','AGROCETEFIDCNP','USECORPCATÁLISEFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSSEGMENTOCOMERCIALDERESPLIMITADA','PAYCARGOFIDCNP',\
                                                             'GREENWOODFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSNAOPADRONIZADOS',\
                                                             'NINE','BONTEMPO','PINPAG','MASTRENN','CREDILOGII-FIDC','CAPITALIZA','F2BANKFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS',\
                                                             'MFGROUPFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOS','UKFFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOS',\
                                                              'BEFICFIDCDERESPONSABILIDADELIMITADA','ANILFUNDODEINVESTIMENTOEMDIREITOSCREDITORIOSDERESPONSABILIDADELIMITADA',\
                                                             'LIBRACONSIGNADOFIDCDERL', 'LEXCAPITALFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA', 'CONDOBEMFUNDODEINVESTIMENTOEMDIREITOSCREDITÓRIOSDERESPONSABILIDADELIMITADA']), arquivos)) 

stop = 0
frames = []

while stop < len(csv_outros):
    file_path = f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados/{csv_outros[stop]}'
    df_csv_outros = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    frames.append(df_csv_outros)
    stop += 1

df_combined_outros = pd.concat(frames, ignore_index=True)


df_csv_outros = df_combined_outros[['FUNDO','CEDENTE','IDENTIFICACAO_CEDENTE','SACADO','IDENTIFICACAO_SACADO',
                                  'SEU_NUMERO','DOCUMENTO','TIPO_RECEBIVEL',
                                  'VL_AQUISICAO','VALOR_PAGO','VALOR_VENCIMENTO','AJUSTE',
                                  'ST_RECEBIVEL','DATA_MOVIMENTO','DATA_AQUISICAO','DATA_VENCIMENTO',
                                  'TX_AQUISICAO','TIPO_MOVIMENTO']]
                                


## renomeia colunas
df_csv_outros.columns = ["NmFundo","NmCedente","DocCedente","NmSacado","DocSacado","Operacao","NuDocumento","TpRecebivel",
                       "VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","SituacaoRecebivel","DtMovimento","DtAquisicao",
                       "DtVencimento","TaxaAquisicao","Ocorrencia"]

date_columns = ['DtMovimento', 'DtVencimento',"DtAquisicao"]
df_csv_outros[date_columns] = df_csv_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

## Tratando dados numéricos

columns_to_convert = ['VlrAquisicao', 'VlrVencimento','VlrPago', 'VlrAjuste','TaxaAquisicao']

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv_outros[column] = df_csv_outros[column].apply(transformar_valor)
    
    
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC SDL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'IPE FIDC NP', 'NmFundo'] = 'FIDC IPE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'K-FINANCE FIDC NP', 'NmFundo'] = 'FIDC K-FINANCE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BLUE ROCKET FIDC', 'NmFundo'] = 'FIDC BLUE ROCKET'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'GLOBAL FUTURA FIDC RL FIDC', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC BELL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'FIDC IPE NP', 'NmFundo'] = 'FIDC IPE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BASA FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BASA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'AF6 FIDC NP', 'NmFundo'] = 'FIDC AF6'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VERGINIA FIDC', 'NmFundo'] = 'FIDC VERGINIA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VISHNU FIDC', 'NmFundo'] = 'FIDC VISHNU'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'DFC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NP', 'NmFundo'] = 'FIDC DFC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'Prime Agro FIDC RL', 'NmFundo'] = 'FIDC PRIME AGRO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'KERDOS FIDC', 'NmFundo'] = 'FIDC KERDOS'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'NmFundo'] = 'FIDC SC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC DBANK'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC FUTURO CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS', 'NmFundo'] = 'FIDC VELSO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'AGROCETE FIDC NP', 'NmFundo'] = 'FIDC AGROCETE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'PAY CARGO FIDC NP', 'NmFundo'] = 'FIDC PAY CARGO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS', 'NmFundo'] = 'FIDC GREENWOOD'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BONTEMPO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'PINPAG FIDC', 'NmFundo'] = 'FIDC PINPAG'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MASTRENN'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CREDILOG II - FIDC', 'NmFundo'] = 'FIDC CREDILOG II'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CAPITALIZA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MF GROUP'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC F2 BANK'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC UKF'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'BEFIC FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BEFIC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANIL'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'LIBRA CONSIGNADO FIDC DE RL', 'NmFundo'] = 'FIDC LIBRA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC MALBEC'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC RHB'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC SMT AGRO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC 3RD'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES', 'NmFundo'] = 'FIDC VITTRA'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANVERES'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC NR11'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TERTON'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TORONTO'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC LEX'
df_csv_outros.loc[df_csv_outros['NmFundo'] == 'CONDOBEM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CONDOBEM'



df_csv_outros[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head()

C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\3924926914.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_outros[date_columns] = df_csv_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\3924926914.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv_outros[column] = df_csv_outros[column].apply(transformar_valor)


,NmFundo,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,TaxaAquisicao
0,FIDC KERDOS,45733.90,47521.38,47521.38,0.0,0.340144
1,FIDC KERDOS,49999.44,50788.03,50788.03,0.0,0.389059
2,FIDC KERDOS,111254.33,113718.64,113718.64,0.0,0.358959
3,FIDC KERDOS,23573.74,24209.29,24209.29,0.0,0.356253
4,FIDC KERDOS,26091.16,29094.79,29094.79,0.0,0.327209


In [21]:
df_csv_outros[['DtMovimento','NmFundo']].drop_duplicates()

,DtMovimento,NmFundo
0,2025-03-20,FIDC KERDOS
23,2025-03-20,FIDC F2 BANK
25,2025-03-20,FIDC CREDILOG II
58,2025-03-20,FIDC CAPITALIZA
60,2025-03-20,FIDC BLUE ROCKET
395,2025-03-20,FIDC BELL
428,2025-03-20,FIDC BEFIC
515,2025-03-20,FIDC BASA
2248,2025-03-20,FIDC ANIL
3442,2025-03-20,FIDC 3RD


_______

In [22]:
print(f"df_LiquidadosBaixados: {len(df_LiquidadosBaixados)}\ndf_csv_agis: {len(df_csv_agis)}\ndf_csv_outros2: {len(df_csv_outros2)} \ndf_csv_outros: {len(df_csv_outros)}")

df_LiquidadosBaixados: 2245
df_csv_agis: 228
df_csv_outros2: 120 
df_csv_outros: 7152


In [23]:
df_final = pd.concat([df_csv_agis, df_csv_outros, df_csv_outros2, df_LiquidadosBaixados], ignore_index=True)

# Filtrando os valores nulos na coluna 'Ocorrencia'
df_nulos_ocorrencia = df_final[df_final['Ocorrencia'].isnull()]
df_nulos_ocorrencia

,NmFundo,NmCedente,DocCedente,NmSacado,DocSacado,Operacao,NuDocumento,TpRecebivel,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,SituacaoRecebivel,DtMovimento,DtAquisicao,DtVencimento,TaxaAquisicao,Ocorrencia


# OPCIONAL

In [24]:
# df_final = pd.concat([df_csv_agis], ignore_index=True)

# Filtrando os valores nulos na coluna 'Ocorrencia'
# df_nulos_ocorrencia = df_final[df_final['Ocorrencia'].isnull()]
# df_nulos_ocorrencia

In [25]:
df_final[["NmFundo"]].drop_duplicates()

,NmFundo
0,FIDC AGIS
228,FIDC KERDOS
251,FIDC F2 BANK
253,FIDC CREDILOG II
286,FIDC CAPITALIZA
288,FIDC BLUE ROCKET
623,FIDC BELL
656,FIDC BEFIC
743,FIDC BASA
2476,FIDC ANIL


In [26]:
df_final[["NmFundo","VlrAquisicao","VlrPago","VlrVencimento","VlrAjuste","TaxaAquisicao"]].head()

,NmFundo,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,TaxaAquisicao
0,FIDC AGIS,7132.45,7364.92,7272.80,-92.12,0.278300
1,FIDC AGIS,6783.98,7005.85,6909.10,-96.75,0.258940
2,FIDC AGIS,81055.63,86622.00,86622.00,0.00,0.334525
3,FIDC AGIS,127423.95,136925.43,136925.43,0.00,0.339514
4,FIDC AGIS,20201.63,21100.00,21100.00,0.00,1.324302


In [27]:
df_final[['DtMovimento','NmFundo']].drop_duplicates().sort_values('DtMovimento')

,DtMovimento,NmFundo
0,2025-03-20,FIDC AGIS
6044,2025-03-20,FIDC NINE CAPITAL
6396,2025-03-20,FIDC MF GROUP
7363,2025-03-20,FIDC MALBEC
7380,2025-03-20,FIDC APG
7417,2025-03-20,FIDC BRISTOL
7500,2025-03-20,FIDC VALENTE
7509,2025-03-20,FIDC TRACTOR
6021,2025-03-20,FIDC NR11
7529,2025-03-20,FIDC TOPCRED


In [28]:
qtdFundo = abs(len(df_final.NmFundo.drop_duplicates()) - len(arquivos))

if qtdFundo == 0:
    print("FALTANTES 0\n")
else:
    print(f"FALTANTES {qtdFundo}\n")
print(f"{df_final.NmFundo.drop_duplicates().sort_values()}")

FALTANTES 0

9733             FIDC 3G BANK
3670                 FIDC 3RD
0                   FIDC AGIS
2476                FIDC ANIL
7380                 FIDC APG
743                 FIDC BASA
656                FIDC BEFIC
623                 FIDC BELL
9681               FIDC BETTA
9608             FIDC BINVEST
288          FIDC BLUE ROCKET
9373                 FIDC BR8
9454              FIDC BRAVOS
7417             FIDC BRISTOL
286           FIDC CAPITALIZA
253          FIDC CREDILOG II
9521                 FIDC DFC
251              FIDC F2 BANK
9559              FIDC FERPAR
9350            FIDC FINTRUST
228               FIDC KERDOS
7363              FIDC MALBEC
6396            FIDC MF GROUP
6044        FIDC NINE CAPITAL
6021                FIDC NR11
9335           FIDC P2 INVEST
5234           FIDC PAY CARGO
5201              FIDC PINPAG
5185          FIDC PRIME AGRO
9255              FIDC PRIMER
5181                 FIDC SDL
3886              FIDC TERTON
7529             FIDC TOPCR

In [29]:
df_final.head()

,NmFundo,NmCedente,DocCedente,NmSacado,DocSacado,Operacao,NuDocumento,TpRecebivel,VlrAquisicao,VlrPago,VlrVencimento,VlrAjuste,SituacaoRecebivel,DtMovimento,DtAquisicao,DtVencimento,TaxaAquisicao,Ocorrencia
0,FIDC AGIS,VIDRACER VIDRADOS CERAMICOS LTDA.,02.115.838/0001-56,ROCHA FORTE COMERCIO E INDUSTRIA LTDA EP,14.256.058/0001-05,201651 00 00 00001,13259/001,Duplicata,7132.45,7364.92,7272.80,-92.12,VENCIDOS,2025-03-20,2025-02-13,2025-03-15,0.278300,0.LIQUIDACAO NORMAL
1,FIDC AGIS,VIDRACER VIDRADOS CERAMICOS LTDA.,02.115.838/0001-56,ROCHA FORTE COMERCIO E INDUSTRIA LTDA EP,14.256.058/0001-05,201181 00 00 00001,13249/001,Duplicata,6783.98,7005.85,6909.10,-96.75,VENCIDOS,2025-03-20,2025-02-11,2025-03-13,0.258940,0.LIQUIDACAO NORMAL
2,FIDC AGIS,TING INDUSTRIA E COMERCIO LTDA,04.827.795/0001-67,COMPANHIA BRASILEIRA DE DISTRIBUICAO - 1,47.508.411/0832-64,257622,00217999/1,Duplicata,81055.63,86622.00,86622.00,0.00,VENCIDOS,2025-03-20,2024-12-23,2025-03-19,0.334525,0.LIQUIDACAO NORMAL
3,FIDC AGIS,TING INDUSTRIA E COMERCIO LTDA,04.827.795/0001-67,SENDAS DISTRIBUIDORA S/A - 60433,06.057.223/0468-30,257491,00217904/1,Duplicata,127423.95,136925.43,136925.43,0.00,VENCIDOS,2025-03-20,2024-12-17,2025-03-19,0.339514,0.LIQUIDACAO NORMAL
4,FIDC AGIS,TRANSHENRY TRANSPORTES LTDA,07.102.721/0001-51,CARRIER REFRIGERACAO BRASIL LTDA,03.646.086/0008-99,203641 00 00 00001,25012/001,CTE Digital,20201.63,21100.00,21100.00,0.00,VENCIDOS,2025-03-20,2025-02-25,2025-03-18,1.324302,0.LIQUIDACAO NORMAL


In [30]:
import mysql.connector
import pandas as pd


def insert_dataframe_to_mysql(df, connection, table_name):
    cursor = connection.cursor()
    for i, row in df.iterrows():
        sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, tuple(row))
    connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_STAGING',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL\n')

    # Dividir o DataFrame em pedaços menores para evitar problemas de memória
    tamanho_do_pedaço = 10000  # Ajuste conforme necessário
    total_registros = len(df_final)

    for i in range(0, total_registros, tamanho_do_pedaço):
        pedaco_df = df_final.iloc[i:i+tamanho_do_pedaço]
        insert_dataframe_to_mysql(pedaco_df, connection, 'Stg_LiquidadosBaixados')
        print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

    print('Concluído')
else:
    print('Falha na conexão ao MySQL')



# Fechar a conexão com o banco de dados
connection.close()

Conectado ao MySQL

Registros inseridos: 9745 / 9745
Concluído


In [31]:
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/LiquidadosBaixados'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

____________

# 

# TRATAMENTO 3 

### Caminho:
- R:\CataliseInvestimentos\14. BigData Catalise\Staging\Operacoes

### Arquivo:
- 20250219_172612_AquisicoesConsolidado_BRISTOLFIDCMULTISSETORIAL.csv

### Tratamento:
1. Alterar de notação ciêntifica para número as colunas (CPF_CNPJ SACADO, CPF_CNPJ_CEDENTE)
2. Alterar coluna "ENTRADA" para data abreviada
3. Alterar formato das colunas "CPF_CNPJSACADO" e "CPF_CNPJ_CEDENTE" para formato CNPJ (00\.000\.000\/0000\-00)

# 

____________

# Operações

In [32]:
import os
import pandas as pd
import time
import pyodbc
from sqlalchemy import create_engine

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

# Obter lista de arquivos
diretorio_operacoes = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Operacoes"
arquivos_ope = os.listdir(diretorio_operacoes)

# Inicializar um DataFrame vazio
df_operacoes_file = pd.DataFrame()

# Iterar sobre os arquivos e anexar ao DataFrame
for arquivo in arquivos_ope:
    file_path = os.path.join(diretorio_operacoes, arquivo)
    
    # Adicionar um bloco try-except para lidar com possíveis erros de leitura
    try:
        df_temp = pd.read_csv(file_path, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
        df_operacoes_file = pd.concat([df_operacoes_file, df_temp], ignore_index=True)
    except pd.errors.EmptyDataError:
        # Lidar com arquivos vazios, se necessário
        print(f"Arquivo vazio: {arquivo}")
    except pd.errors.ParserError as e:
        # Lidar com erros de leitura, se necessário
        print(f"Erro na leitura do arquivo {arquivo}: {e}")
        
# df_operacoes_file = df_operacoes_file.drop_duplicates()
        
## 1 MASTER
df_operacoes = df_operacoes_file.query("FUNDO not in ['AGIS FIDC NAO PADRONIZADO - TRANSFERIDO','ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
                                       'AGROFORTE FIDC','AGIS FIDC NAO PADRONIZADO','APG FIDC',\
                                       'SC FIDC RESPONSABILIDADE ILIMITADA', 'IPE FIDC NP', 'K-FINANCE FIDC NP',\
                                       'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','BLUE ROCKET FIDC',\
                                       'Z INVEST FIDC','GLOBAL FUTURA FIDC RL FIDC','BRISTOL FIDC MULTISSETORIAL','3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','BASA FIDC DE RESPONSABILIDADE LIMITADA',\
                                       'FIDC UKF', 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'ALBAREDO FIDC',\
                                       'Prime Agro FIDC RL','ÁGIS FIDC','SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA','VERGINIA FIDC','MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','CREDIAL BANK PAN FIDC',\
                                       'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
                                       'PINPAG FIDC','KERDOS FIDC','YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS','PAY CARGO FIDC NP',\
                                       'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
                                       'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS','TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','BEFIC FIDC DE RESPONSABILIDADE LIMITADA',\
                                       'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','AF6 FIDC NP','LIBRA CONSIGNADO FIDC DE RL',\
                                       'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','CREDILOG II - FIDC',\
                                       'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
                                       'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',\
                                       'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
                                       'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES',\
                                       'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA']")



## 3 DAYCOVAL
df_operacoes_agis2 = df_operacoes_file.query("FUNDO in ['AGIS FIDC NAO PADRONIZADO - TRANSFERIDO','ÁGIS FIDC','ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','SC FIDC RESPONSABILIDADE ILIMITADA','K-FINANCE FIDC NP',\
'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','BLUE ROCKET FIDC','Z INVEST FIDC','GLOBAL FUTURA FIDC RL FIDC',\
'BASA FIDC DE RESPONSABILIDADE LIMITADA','FIDC UKF','YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA','MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'IPE FIDC NP','3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'ALBAREDO FIDC',\
'VERGINIA FIDC','CREDIAL BANK PAN FIDC','BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
'PINPAG FIDC','Prime Agro FIDC RL','KERDOS FIDC','TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA','FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS','PAY CARGO FIDC NP','AGROCETE FIDC NP','RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA',\
'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS','BEFIC FIDC DE RESPONSABILIDADE LIMITADA',\
'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS','AF6 FIDC NP','LIBRA CONSIGNADO FIDC DE RL','MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',\
'CREDILOG II - FIDC', 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',\
'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS','NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA',\
'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA','VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES',\
'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'EJM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA']")

## ALTERA NOME
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'FUNDO'] = 'FIDC SC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC SDL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'K-FINANCE FIDC NP', 'FUNDO'] = 'FIDC K-FINANCE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BLUE ROCKET FIDC', 'FUNDO'] = 'FIDC BLUE ROCKET'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'GLOBAL FUTURA FIDC RL FIDC', 'FUNDO'] = 'FIDC GLOBAL FUTURA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BASA FIDC DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BASA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'IPE FIDC NP', 'FUNDO'] = 'FIDC IPE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC BELL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VERGINIA FIDC', 'FUNDO'] = 'FIDC VERGINIA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CREDIAL BANK PAN FIDC', 'FUNDO'] = 'FIDC CREDIAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BONTEMPO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC DBANK'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'PINPAG FIDC', 'FUNDO'] = 'FIDC PINPAG'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'Prime Agro FIDC RL', 'FUNDO'] = 'FIDC PRIME AGRO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'KERDOS FIDC', 'FUNDO'] = 'FIDC KERDOS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC FUTURO CAPITAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS', 'FUNDO'] = 'FIDC VELSO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'PAY CARGO FIDC NP', 'FUNDO'] = 'FIDC PAY CARGO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AGROCETE FIDC NP', 'FUNDO'] = 'FIDC AGROCETE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC CAPITALIZA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS', 'FUNDO'] = 'FIDC GREENWOOD'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BEFIC FIDC DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC BEFIC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'AF6 FIDC NP', 'FUNDO'] = 'FIDC AF6'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'LIBRA CONSIGNADO FIDC DE RL', 'FUNDO'] = 'FIDC LIBRA CONSIGNADO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC MASTRENN'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'CREDILOG II - FIDC', 'FUNDO'] = 'FIDC CREDILOG II'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC NINE CAPITAL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC UKF'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC MF GROUP'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'FUNDO'] = 'FIDC F2 BANK'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC NR11'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ÁGIS FIDC', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ALBAREDO FIDC', 'FUNDO'] = 'FIDC ALBAREDO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'Z INVEST FIDC', 'FUNDO'] = 'FIDC Z INVEST'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC USECORP CATALISE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ANVERES'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES', 'FUNDO'] = 'FIDC VITTRA'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ANIL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC 3RD'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'FUNDO'] = 'FIDC MALBEC'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA', 'FUNDO'] = 'FIDC USECORP CATALISE'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC RHB'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC SMT AGRO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC TERTON'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC TORONTO'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC YELLOW'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC LEX'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'BRISTOL FIDC MULTISSETORIAL', 'FUNDO'] = 'FIDC BRISTOL'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'EJM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC EJM'
df_operacoes_agis2.loc[df_operacoes_agis2['FUNDO'] == 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC SAGEL BANK'



## ALTERA NOME
## 2
df_operacoes_outros = df_operacoes_file.query("FUNDO in ['AGROFORTE FIDC','AGIS FIDC NAO PADRONIZADO','APG FIDC', 'BRISTOL FIDC MULTISSETORIAL']")

## ALTERA NOME
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'AGIS FIDC NAO PADRONIZADO', 'FUNDO'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'ÁGIS FIDC', 'FUNDO'] = 'FIDC AGIS'

df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'AGROFORTEFIDC', 'FUNDO'] = 'FIDC AGROFORTE'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'APG FIDC', 'FUNDO'] = 'FIDC APG'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'BRISTOL FIDC MULTISSETORIAL', 'FUNDO'] = 'FIDC BRISTOL'
df_operacoes_outros.loc[df_operacoes_outros['FUNDO'] == 'ZAB LEGACY FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'FUNDO'] = 'FIDC ZAB LEGACY'

print(f" df_operacoes: {len(df_operacoes)}\n df_operacoes_outros: {len(df_operacoes_outros)}\n df_operacoes_agis2: {len(df_operacoes_agis2)}")

df1 = df_operacoes[['FUNDO']].drop_duplicates()
df2 = df_operacoes_agis2[['FUNDO']].drop_duplicates()
df3 = df_operacoes_outros[['FUNDO']].drop_duplicates()

fundos_concatenados = pd.concat([df1, df2, df3]).drop_duplicates()


df = len(fundos_concatenados)

qtdFundo = abs(df - len(arquivos_ope))

if qtdFundo == 0:
    print("FALTANTES 0\n")      
else:
    print(f"FALTANTES {qtdFundo}\n")

 df_operacoes: 3236
 df_operacoes_outros: 116
 df_operacoes_agis2: 10092
FALTANTES 0



In [33]:
#df_operacoes[['FUNDO', 'ENTRADA']].drop_duplicates()
#df_operacoes_agis2[['ENTRADA','FUNDO']].drop_duplicates()
df_operacoes_outros[['ENTRADA','FUNDO']].drop_duplicates()

,ENTRADA,FUNDO
0,2025-03-20,FIDC APG
107,20/03/2025,FIDC BRISTOL


In [34]:
df_operacoes = df_operacoes[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]

# df_operacoes.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','VlrPago','VlrPresente','VlrAjuste','DtVencimento','DtAquisicao']
df_operacoes.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']

date_columns = ['DtReferencia', 'DtVencimento']
df_operacoes[date_columns] = df_operacoes[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors='coerce')

## Tratando dados numéricos

columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes[column] = df_operacoes[column].apply(transformar_valor)

df_operacoes = df_operacoes.sort_values(by='DtReferencia',ascending=True)


In [35]:
df_operacoes[['VlrNominal', 'VlrAquisicao']].head()

,VlrNominal,VlrAquisicao
116,219127.60,209800.58
2267,199.00,85.23
2268,234.43,100.40
2269,295.56,126.58
2270,202.65,86.79


In [36]:
df_operacoes_outros.head()

,FUNDO,CEDENTE,ENTRADA,DATA VENCIMENTO,QUANTIDADE,VALOR DE COMPRA,VALOR DE VENCIMENTO,TIPO RECEBIVEL,ID RECEBIVEL,NUMERO DOCUMENTO,...,NOME SACADO,CPF_CNPJ SACADO,CPF_CNPJ_CEDENTE,Unnamed: 14,ID_FUNDO,ID_CEDENTE,ID LOTE,ID OPERAÇÃO BANCO,NOSSO NUMERO,COOBRIGACAO
0,FIDC APG,CASEX IND DE PLASTICOS E PROD MEDICOS HOSPITAL...,2025-03-20,2025-04-22,1,8628.04,9033.33,Duplicata,341338618,36225001,...,JOAOMED COMERCIO DE MATERIAIS CIRURGICOS,78742491000133,78746773000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FIDC APG,CASEX IND DE PLASTICOS E PROD MEDICOS HOSPITAL...,2025-03-20,2025-04-22,1,5259.15,5506.20,Duplicata,341338619,36226001,...,GEMED COMERCIO DE ARTIGOS MEDICOS E ORTO,08211975000170,78746773000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FIDC APG,CASEX IND DE PLASTICOS E PROD MEDICOS HOSPITAL...,2025-03-20,2025-04-29,1,8535.39,9033.33,Duplicata,341338620,36225002,...,JOAOMED COMERCIO DE MATERIAIS CIRURGICOS,78742491000133,78746773000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FIDC APG,CASEX IND DE PLASTICOS E PROD MEDICOS HOSPITAL...,2025-03-20,2025-04-29,1,5202.69,5506.20,Duplicata,341338621,36226002,...,GEMED COMERCIO DE ARTIGOS MEDICOS E ORTO,08211975000170,78746773000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FIDC APG,CASEX IND DE PLASTICOS E PROD MEDICOS HOSPITAL...,2025-03-20,2025-05-06,1,8465.92,9033.34,Duplicata,341338622,36225003,...,JOAOMED COMERCIO DE MATERIAIS CIRURGICOS,78742491000133,78746773000109,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
print(df_operacoes[['NmFundo','DtReferencia']].drop_duplicates().head(300).to_string())

             NmFundo DtReferencia
116     FIDC TRACTOR   2025-03-20
2267    FIDC TOPCRED   2025-03-20
3076        BR8 FIDC   2025-03-20
2975   FIDC FINTRUST   2025-03-20
2994     FIDC FERPAR   2025-03-20
3026        FIDC DFC   2025-03-20
3278      FIDC BETTA   2025-03-20
3329    FIDC 3G BANK   2025-03-20
3193    FIDC BINVEST   2025-03-20
2874  FIDC P2 INVEST   2025-03-20
2849     FIDC PRIMER   2025-03-20
9249     VISHNU FIDC   2025-03-20


In [38]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('ZAB LEGACY FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', case=False, na=False), 
    'NmFundo'
] = 'FIDC ZAB LEGACY'

In [39]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('BR8 FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC BR8'

In [40]:
df_operacoes.loc[
    df_operacoes['NmFundo'].str.contains('VISHNU FIDC', case=False, na=False), 
    'NmFundo'
] = 'FIDC VISHNU'

In [41]:
df_operacoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3236 entries, 116 to 9249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   NmFundo       3236 non-null   object        
 1   DtReferencia  3236 non-null   datetime64[ns]
 2   DocCedente    3236 non-null   object        
 3   DocSacado     3236 non-null   object        
 4   Operacao      3236 non-null   object        
 5   NuDocumento   3236 non-null   object        
 6   TpRecebivel   3236 non-null   object        
 7   VlrNominal    3236 non-null   float64       
 8   VlrAquisicao  3236 non-null   float64       
 9   DtVencimento  3236 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(6)
memory usage: 278.1+ KB


Mais rápido

In [42]:
import mysql.connector
import pandas as pd
import time
start_time = time.time()

df_final = df_operacoes
 
# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    df_operacoes.fillna(0, inplace=True)
    insert_dataframe_to_mysql_batch(df_operacoes, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

Conectado ao MySQL
Concluído
Tempo decorrido: 0.6355423927307129 segundos


In [43]:
# Contar valores NaN em cada coluna
nan_por_coluna = df_operacoes.isna().sum()

# Exibir o resumo
print(nan_por_coluna)

NmFundo         0
DtReferencia    0
DocCedente      0
DocSacado       0
Operacao        0
NuDocumento     0
TpRecebivel     0
VlrNominal      0
VlrAquisicao    0
DtVencimento    0
dtype: int64


In [44]:
df_operacoes.fillna(0, inplace=True)

# outros

In [45]:
import time

import pyodbc
from sqlalchemy import create_engine


df_operacoes_outros = df_operacoes_outros[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]


df_operacoes_outros.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']


# Função para converter datas com base no nome do fundo
def converter_datas(row):
    if row['NmFundo'] in ['FIDC AGROFORTE', 'FIDC APG']:
        try:
            row['DtReferencia'] = pd.to_datetime(row['DtReferencia'], format='%Y-%m-%d', errors='coerce')
            row['DtVencimento'] = pd.to_datetime(row['DtVencimento'], format='%Y-%m-%d', errors='coerce')
        except:
            row['DtReferencia'] = pd.NaT
            row['DtVencimento'] = pd.NaT
            
    elif row['NmFundo'] == 'FIDC BRISTOL':
        try:
            row['DtReferencia'] = pd.to_datetime(row['DtReferencia'], format='%d/%m/%Y', errors='coerce')
            row['DtVencimento'] = pd.to_datetime(row['DtVencimento'], format='%d/%m/%Y', errors='coerce')
        except:
            row['DtReferencia'] = pd.NaT
            row['DtVencimento'] = pd.NaT
            
    return row

# Aplicando a função de conversão ao DataFrame
df_operacoes_outros = df_operacoes_outros.apply(converter_datas, axis=1)

# Garantindo que todas as datas estejam no formato Timestamp
df_operacoes_outros['DtReferencia'] = pd.to_datetime(df_operacoes_outros['DtReferencia'], errors='coerce')
df_operacoes_outros['DtVencimento'] = pd.to_datetime(df_operacoes_outros['DtVencimento'], errors='coerce')

## Tratando dados numéricos
columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes_outros[column] = df_operacoes_outros[column].apply(transformar_valor)
    
df_operacoes_outros['VlrNominal'] = df_operacoes_outros['VlrNominal']/100
df_operacoes_outros['VlrAquisicao'] = df_operacoes_outros['VlrAquisicao']/100

# Ajustando nomes do fundos
df_operacoes_outros = df_operacoes_outros.sort_values(by='DtReferencia',ascending=True)
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO', 'NmFundo'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGROFORTE FIDC', 'NmFundo'] = 'FIDC AGROFORTE'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'APG FIDC', 'NmFundo'] = 'FIDC APG'
df_operacoes_outros.NmFundo.drop_duplicates()

0          FIDC APG
113    FIDC BRISTOL
Name: NmFundo, dtype: object

In [46]:
df_operacoes_outros[['NmFundo','DtReferencia','DtVencimento','VlrNominal','VlrAquisicao']]

,NmFundo,DtReferencia,DtVencimento,VlrNominal,VlrAquisicao
0,FIDC APG,2025-03-20,2025-04-22,9033.33,8628.04
84,FIDC APG,2025-03-20,2025-04-16,473.80,455.75
83,FIDC APG,2025-03-20,2025-04-16,224.20,215.66
82,FIDC APG,2025-03-20,2025-04-16,202.30,194.59
81,FIDC APG,2025-03-20,2025-04-16,1146.64,1102.95
...,...,...,...,...,...
32,FIDC APG,2025-03-20,2025-06-28,5957.02,5384.09
31,FIDC APG,2025-03-20,2025-06-25,4445.06,4038.10
30,FIDC APG,2025-03-20,2025-06-25,6237.29,5666.24
42,FIDC APG,2025-03-20,2025-07-12,2004.66,1785.90


In [47]:
df_operacoes_outros

,NmFundo,DtReferencia,DocCedente,DocSacado,Operacao,NuDocumento,TpRecebivel,VlrNominal,VlrAquisicao,DtVencimento
0,FIDC APG,2025-03-20,78746773000109,78742491000133,0000000388510000000014271,36225001,Duplicata,9033.33,8628.04,2025-04-22
84,FIDC APG,2025-03-20,11801723000106,08283683000143,0000000442900000000014231,0000055121,Duplicata,473.80,455.75,2025-04-16
83,FIDC APG,2025-03-20,11801723000106,10247895000117,0000000442900000000014220,0000055111,Duplicata,224.20,215.66,2025-04-16
82,FIDC APG,2025-03-20,11801723000106,01970368000145,0000000442900000000014233,0000055091,Duplicata,202.30,194.59,2025-04-16
81,FIDC APG,2025-03-20,11801723000106,76071745000103,0000000442900000000014225,0000055081,Duplicata,1146.64,1102.95,2025-04-16
...,...,...,...,...,...,...,...,...,...,...
32,FIDC APG,2025-03-20,16588294000154,31770650000140,0000000499220000000013816,402/21,Contrato Fisico,5957.02,5384.09,2025-06-28
31,FIDC APG,2025-03-20,16588294000154,04584925909,0000000499220000000013809,802/70,Contrato Fisico,4445.06,4038.10,2025-06-25
30,FIDC APG,2025-03-20,16588294000154,10823907000295,0000000499220000000013795,701/53,Contrato Fisico,6237.29,5666.24,2025-06-25
42,FIDC APG,2025-03-20,16588294000154,00878372008,0000000499220000000013860,601/5,Contrato Fisico,2004.66,1785.90,2025-07-12


In [48]:
# df_operacoes_outros = df_operacoes_outros.query("DtReferencia=='2024-07-08'")

In [49]:
import mysql.connector
import pandas as pd
import time
start_time = time.time()

df_final = df_operacoes_outros
 
# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    insert_dataframe_to_mysql_batch(df_final, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

Conectado ao MySQL
Concluído
Tempo decorrido: 0.21124911308288574 segundos


_________

# Daycoval


In [50]:
import time

import pyodbc
from sqlalchemy import create_engine


df_operacoes_outros = df_operacoes_agis2[['FUNDO',
                            'ENTRADA',
                            'CPF_CNPJ_CEDENTE',
                            'CPF_CNPJ SACADO' ,
                            'SEU NUMERO' ,
                            'NUMERO DOCUMENTO' ,
                            'TIPO RECEBIVEL' ,
                            'VALOR DE VENCIMENTO',
                            'VALOR DE COMPRA' ,
                            'DATA VENCIMENTO']]


df_operacoes_outros.columns = ['NmFundo','DtReferencia','DocCedente','DocSacado','Operacao','NuDocumento','TpRecebivel','VlrNominal','VlrAquisicao','DtVencimento']

date_columns = ['DtReferencia', 'DtVencimento']
df_operacoes_outros[date_columns] = df_operacoes_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors='coerce')

## Tratando dados numéricos

columns_to_convert = ['VlrNominal', 'VlrAquisicao']

for column in columns_to_convert:
    df_operacoes_outros[column] = df_operacoes_outros[column].apply(transformar_valor)

df_operacoes_outros = df_operacoes_outros.sort_values(by='DtReferencia',ascending=True)

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC USECORP CATALISE SR'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'ÁGIS FIDC', 'NmFundo'] = 'FIDC AGIS'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'

# INVESTIGAR ALBAREDO

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'NmFundo'] = 'FIDC SC'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'IPE FIDC NP', 'NmFundo'] = 'FIDC IPE'

df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'BLUE ROCKET FIDC', 'NmFundo'] = 'FIDC BLUE ROCKET'
df_operacoes_outros.loc[df_operacoes_outros['NmFundo'] == 'FIDC BASA', 'NmFundo'] = 'FIDC BASA'


C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\3018769356.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operacoes_outros[date_columns] = df_operacoes_outros[date_columns].apply(pd.to_datetime, format="%d/%m/%Y", errors='coerce')
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_2728\3018769356.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operacoes_outros[column] = df_operacoes_outros[column].apply(transformar_valor)


In [51]:
df_operacoes_outros.NmFundo.drop_duplicates()

3351               FIDC KERDOS
10076              FIDC TERTON
10881                 FIDC SDL
8814               FIDC VITTRA
9367     FIDC USECORP CATALISE
9382              FIDC TORONTO
12599            FIDC MF GROUP
11338           FIDC PAY CARGO
11354                FIDC NR11
11495        FIDC NINE CAPITAL
11022              FIDC PINPAG
5028                 FIDC BASA
4634                FIDC BEFIC
3767              FIDC F2 BANK
3815                FIDC DBANK
3924          FIDC BLUE ROCKET
3904          FIDC CREDILOG II
7552                 FIDC ANIL
8183                  FIDC 3RD
7925                 FIDC AGIS
Name: NmFundo, dtype: object

In [52]:
df_operacoes_outros[['VlrNominal', 'VlrAquisicao','DocCedente', 'DocSacado']]

,VlrNominal,VlrAquisicao,DocCedente,DocSacado
3351,261304.84,258660.37,77.887.917/0001-84,18.627.195/0001-60
10076,331.76,319.47,00.502.754/0006-54,31.638.145/0001-46
10077,557.65,536.99,00.502.754/0006-54,05.004.708/0001-34
10078,165.67,159.53,00.502.754/0006-54,106.378.409-33
10079,386.55,372.23,00.502.754/0006-54,13.983.315/0001-48
...,...,...,...,...
6716,514.23,501.56,04.310.364/0001-29,18.490.749/0001-20
6717,877.51,855.90,04.310.364/0001-29,01.793.285/0001-28
6718,542.91,529.54,04.310.364/0001-29,01.793.285/0001-28
6711,937.52,914.43,04.310.364/0001-29,82.117.144/0001-97


In [53]:
df_operacoes_outros[['DtReferencia', 'NmFundo']].drop_duplicates()

,DtReferencia,NmFundo
3351,2025-03-20,FIDC KERDOS
10076,2025-03-20,FIDC TERTON
10881,2025-03-20,FIDC SDL
8814,2025-03-20,FIDC VITTRA
9367,2025-03-20,FIDC USECORP CATALISE
9382,2025-03-20,FIDC TORONTO
12599,2025-03-20,FIDC MF GROUP
11338,2025-03-20,FIDC PAY CARGO
11354,2025-03-20,FIDC NR11
11495,2025-03-20,FIDC NINE CAPITAL


In [54]:
df_final

,NmFundo,DtReferencia,DocCedente,DocSacado,Operacao,NuDocumento,TpRecebivel,VlrNominal,VlrAquisicao,DtVencimento
0,FIDC APG,2025-03-20,78746773000109,78742491000133,0000000388510000000014271,36225001,Duplicata,9033.33,8628.04,2025-04-22
84,FIDC APG,2025-03-20,11801723000106,08283683000143,0000000442900000000014231,0000055121,Duplicata,473.80,455.75,2025-04-16
83,FIDC APG,2025-03-20,11801723000106,10247895000117,0000000442900000000014220,0000055111,Duplicata,224.20,215.66,2025-04-16
82,FIDC APG,2025-03-20,11801723000106,01970368000145,0000000442900000000014233,0000055091,Duplicata,202.30,194.59,2025-04-16
81,FIDC APG,2025-03-20,11801723000106,76071745000103,0000000442900000000014225,0000055081,Duplicata,1146.64,1102.95,2025-04-16
...,...,...,...,...,...,...,...,...,...,...
32,FIDC APG,2025-03-20,16588294000154,31770650000140,0000000499220000000013816,402/21,Contrato Fisico,5957.02,5384.09,2025-06-28
31,FIDC APG,2025-03-20,16588294000154,04584925909,0000000499220000000013809,802/70,Contrato Fisico,4445.06,4038.10,2025-06-25
30,FIDC APG,2025-03-20,16588294000154,10823907000295,0000000499220000000013795,701/53,Contrato Fisico,6237.29,5666.24,2025-06-25
42,FIDC APG,2025-03-20,16588294000154,00878372008,0000000499220000000013860,601/5,Contrato Fisico,2004.66,1785.90,2025-07-12


In [55]:
# Contar valores NaN em cada coluna
nan_por_coluna = df_final.isna().sum()

# Exibir o resumo
print(nan_por_coluna)

NmFundo         0
DtReferencia    0
DocCedente      0
DocSacado       0
Operacao        0
NuDocumento     0
TpRecebivel     0
VlrNominal      0
VlrAquisicao    0
DtVencimento    0
dtype: int64


In [56]:
df_final.fillna(0, inplace=True)

In [57]:
import mysql.connector
import pandas as pd
import time

start_time = time.time()

df_final = df_operacoes_outros

# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    # Preencher valores NaN com uma string vazia
    df = df.fillna('')

    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    insert_dataframe_to_mysql_batch(df_final, connection, 'Ft_Operacoes', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

Conectado ao MySQL
Concluído
Tempo decorrido: 1.3557801246643066 segundos


In [58]:
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Operacoes'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))